In [ ]:
import datetime
import glob
import numpy as np
import xarray as xr
import rasterio
import rioxarray
import pandas as pd
from matplotlib import pyplot
from rasterio.warp import Resampling

In [ ]:
with rasterio.open('data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h15v05_2023.tiff') as dataset:
    lc_map = dataset.read(1)

In [ ]:
pyplot.imshow(lc_map, interpolation = 'nearest')

In [ ]:
file_list = glob.glob('data/processed/VNP16_ET_and_PET/VNP16_ET_mm_8day-1_*.tiff')
file_list.sort()

rasters_et = []
rasters_pet = []
for filename in file_list:
    date_str = filename.split('/')[-1].split('_')[-1].replace('.tiff', '')
    date = datetime.datetime.strptime(date_str, '%Y%m%d')
    et0 = rioxarray.open_rasterio(filename)\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    rasters_et.append(et0)
    # Do the same for PET, note the use of replace()
    pet0 = rioxarray.open_rasterio(filename.replace('ET_mm_8day-1', 'PET_mm_8day-1'))\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    rasters_pet.append(pet0)

# Convert from [mm 8day-1] to [mm day-1]
ds_et0 = xr.concat(rasters_et, dim = 'time').to_dataset(name = 'ET') / 8.0
ds_pet0 = xr.concat(rasters_pet, dim = 'time').to_dataset(name = 'PET') / 8.0

ds_et = xr.merge([ds_et0, ds_pet0])

```
# TODO Resampling to daily time steps; we have to a

ds_et_last = ds_et.isel(time = -1).assign_coords(time = pd.Timestamp('2023-12-31'))
ds_et_daily = xr.concat([ds_et, ds_et_last], dim = 'time')\
    .resample(time = '1D', skipna = False).ffill()
ds_et_daily
```

## Computing the initial soil water content

In [ ]:
# TODO Now loading precipitation
example = rioxarray.open_rasterio('data/processed/VNP16_ET_and_PET/VNP16_ET_mm_8day-1_20230101.tiff')

# Resample the precip data to match our ET data
ds_precip = xr.open_dataset('data/processed/IMERG_precip_mm_day-1_for_study_area.nc4')\
    .rio.write_crs(example.rio.crs)
ds_precip_500m = ds_precip.rio.reproject_match(example, resampling = Resampling.bilinear)

In [ ]:
precip_total = ds_precip_500m.sum('time').precipitation

In [ ]:
# NOTE: Our best guess for the soil water content might be
#    the mean annual balance between precip and ET

swc = (precip_total - ds_et.ET.sum('time')) / 365

In [ ]:
swc.where(swc >= 0, np.nan).where(lc_map == 1, np.nan).plot()

## Field capacity, rooting depth

The critical soil water level (SWC) is given by:
$$
\text{SWC} = \text{FC}\times \text{SW}_f\times \text{RD}_f
$$

- Field capacity (FC) depends on soil texture; we could use a texture map to account for the heterogeneity across our study area; for simplicity, we'll assume the soils are a sandy loam, with a median field capacity (from FAO 56 Table 19) of 0.23 m$^3$ m$^{-3}$. If the maximum rooting depth of wheat is about 1.5 m (https://doi.org/10.1016/j.fcr.2016.02.013), then this corresponds to:
$$
\text{FC} = \frac{0.23\, \text{m}^3\text{m}^{-3}}{1.5\text{m}\times 1.0\text{m} \times 1.0\text{m}} = 0.153 \text{m}
$$
- According to FAO 56 (Table 22), for Spring Wheat, $\text{SW}_f = 1 - 0.55 = 0.45$.
- According to FAO 56 (Table 22), for Spring Wheat, 
- Rooting depth depends primarily on the type of crop present

In [ ]:
# Convert field capacity from VWC to mm of water
0.23

## Our first implementation

In [ ]:
def water_requirement_satisfaction_index(sw_init, precip_t, field_capacity, rooting_depth, crop_coef = None):
    # At t=0, use sw_init in Equations 3-6 to compute AET
    # Then, for t >= 0:
    #    Compute SW by Equation 8 (i.e., use AET just calculated)
    pass